# 🏛️ Legal Knowledge Graph v2 - Google Colab Edition

한국어 법률 문서를 Google Gemini API를 활용하여 지식그래프로 변환하는 노트북입니다.

## 📋 주요 기능
- ✅ GPU 가속 지원 (T4)
- ✅ Google Gemini API 연동
- ✅ PDF 텍스트 추출 (PyMuPDF)
- ✅ 법률 조항 자동 분리
- ✅ 지식 그래프 생성
- ✅ 결과 JSON 저장 및 다운로드
- ✅ 한글 지원

## 1️⃣ 환경 설정 및 설치

In [ ]:
# GPU 확인
import subprocess
import sys

try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print("✅ GPU 사용 가능:")
    print(result.stdout)
except FileNotFoundError:
    print("⚠️ GPU를 사용할 수 없습니다. CPU 모드로 실행됩니다.")

In [ ]:
# 저장소 클론
!git clone https://github.com/WB-Jang/Knowledge-graph-construction-LLM-v2.git
%cd Knowledge-graph-construction-LLM-v2

In [ ]:
# 필요한 패키지 설치
!pip install -q langchain langchain-community langchain-google-genai langgraph
!pip install -q google-generativeai langchain-core
!pip install -q pydantic python-dotenv
!pip install -q rich tqdm
!pip install -q PyMuPDF

print("✅ 패키지 설치 완료")

In [ ]:
# 한글 폰트 설정 (시각화용)
!apt-get install -qq fonts-nanum
!fc-cache -fv

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 나눔 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False

print("✅ 한글 폰트 설정 완료")

## 2️⃣ API 키 설정

Google AI Studio에서 Gemini API 키를 발급받으세요: https://makersuite.google.com/app/apikey

In [ ]:
import os
from getpass import getpass

# Gemini API 키 입력
api_key = getpass("Gemini API 키를 입력하세요: ")
os.environ["GOOGLE_API_KEY"] = api_key
os.environ["USE_LOCAL_LLM"] = "false"
os.environ["GEMINI_MODEL"] = "gemini-2.0-flash-exp"
os.environ["LLM_TEMPERATURE"] = "0.0"
os.environ["LLM_MAX_TOKENS"] = "2048"

print("✅ API 키 설정 완료")

## 3️⃣ Colab용 간소화 스크립트 생성

Colab 환경에서는 Memgraph를 사용할 수 없으므로, 메모리 기반으로 처리하는 간소화된 버전을 생성합니다.

In [ ]:
# Colab용 간소화 스크립트 생성
colab_script = '''
"""Colab용 간소화 실행 스크립트"""
import os
import sys
import json
from pathlib import Path
from typing import List

# 프로젝트 경로 추가
sys.path.insert(0, "/content/Knowledge-graph-construction-LLM-v2/src")

from models.schemas import LegalDocument, LegalEntity, GraphTriplet
from graphs.legal_graph import LegalKnowledgeGraphWorkflow
from utils.text_processor import split_articles
from rich.console import Console
from rich.table import Table

console = Console()

def process_document(title: str, law_number: str, content: str, max_articles: int = 3) -> LegalDocument:
    """문서를 처리하여 지식 그래프를 생성합니다."""
    console.print(f"\n🚀 [{title}] 처리 시작...", style="bold green")
    
    # 조항 분리
    articles = split_articles(content)
    console.print(f"📄 발견된 조항: {len(articles)}개")
    
    # Colab에서는 처음 max_articles개만 처리 (API 제한 고려)
    if len(articles) > max_articles:
        console.print(f"⚠️ Colab 환경: 처음 {max_articles}개 조항만 처리합니다", style="yellow")
        content_limited = "\n\n".join(articles[:max_articles])
    else:
        content_limited = content
    
    document = LegalDocument(
        title=title,
        law_number=law_number,
        content=content_limited
    )
    
    # 워크플로우 실행
    workflow = LegalKnowledgeGraphWorkflow()
    
    with console.status("[bold green]처리 중...", spinner="dots"):
        result = workflow.process(document)
    
    console.print(f"✅ 처리 완료!", style="bold green")
    console.print(f"   추출된 개체: {len(result.entities)}개")
    console.print(f"   추출된 관계: {len(result.triplets)}개")
    
    return result

def display_results(document: LegalDocument):
    """결과를 테이블로 표시합니다."""
    # 개체 테이블
    if document.entities:
        entity_table = Table(title=f"📊 추출된 개체 ({len(document.entities)}개)")
        entity_table.add_column("조항", style="cyan")
        entity_table.add_column("개념", style="magenta")
        entity_table.add_column("주체", style="green")
        entity_table.add_column("행위", style="yellow")
        
        for entity in document.entities[:10]:
            entity_table.add_row(
                entity.article_number,
                entity.concept[:30],
                entity.subject or "-",
                entity.action or "-"
            )
        
        console.print(entity_table)
    
    # 관계 테이블
    if document.triplets:
        relation_table = Table(title=f"🔗 추출된 관계 ({len(document.triplets)}개)")
        relation_table.add_column("주체", style="cyan")
        relation_table.add_column("관계", style="magenta")
        relation_table.add_column("대상", style="green")
        relation_table.add_column("신뢰도", style="yellow")
        
        for triplet in document.triplets[:10]:
            relation_table.add_row(
                triplet.subject[:20],
                triplet.relation,
                triplet.object[:20],
                f"{triplet.confidence:.2f}"
            )
        
        console.print(relation_table)

def save_to_json(document: LegalDocument, output_path: str = "result.json"):
    """결과를 JSON 파일로 저장합니다."""
    data = {
        "title": document.title,
        "law_number": document.law_number,
        "entities": [entity.model_dump() for entity in document.entities],
        "triplets": [triplet.model_dump() for triplet in document.triplets]
    }
    
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    console.print(f"\n💾 결과 저장 완료: {output_path}", style="bold green")
    return output_path
'''

# 스크립트 파일 생성
with open('colab_runner.py', 'w', encoding='utf-8') as f:
    f.write(colab_script)

print("✅ Colab용 실행 스크립트 생성 완료")

## 4️⃣ 샘플 실행 (개인정보보호법)

샘플 법률 텍스트로 지식 그래프를 생성합니다.

In [ ]:
# Colab 러너 임포트
import sys
sys.path.insert(0, '/content/Knowledge-graph-construction-LLM-v2/src')

from colab_runner import process_document, display_results, save_to_json

# 샘플 법률 문서
sample_content = """
제1조(목적) 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인의 존엄과 가치를 구현함을 목적으로 한다.

제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다.
1. "개인정보"란 살아 있는 개인에 관한 정보로서 성명, 주민등록번호 및 영상 등을 통하여 개인을 알아볼 수 있는 정보를 말한다.

제3조(개인정보 보호 원칙) ① 개인정보처리자는 개인정보의 처리 목적을 명확하게 하여야 하고 그 목적에 필요한 범위에서 최소한의 개인정보만을 적법하고 정당하게 수집하여야 한다.
""".strip()

# 문서 처리 (처음 3개 조항만)
result = process_document(
    title="개인정보 보호법",
    law_number="법률 제18583호",
    content=sample_content,
    max_articles=3
)

# 결과 표시
display_results(result)

# JSON으로 저장
json_path = save_to_json(result, "sample_result.json")

print("\n" + "="*80)
print("✨ 샘플 처리 완료!")
print("="*80)

## 5️⃣ PDF 파일 처리

PDF 파일을 업로드하여 처리할 수 있습니다.

In [ ]:
# PDF 파일 업로드
from google.colab import files
import os

print("📁 PDF 파일을 업로드하세요...")
uploaded = files.upload()

if uploaded:
    pdf_filename = list(uploaded.keys())[0]
    print(f"✅ 업로드 완료: {pdf_filename}")
else:
    print("❌ 파일이 업로드되지 않았습니다.")
    pdf_filename = None

In [ ]:
# PDF 처리
if pdf_filename:
    import sys
    sys.path.insert(0, '/content/Knowledge-graph-construction-LLM-v2/src')
    
    from utils.pdf_processor import extract_text_from_pdf, get_pdf_metadata
    from pathlib import Path
    from colab_runner import process_document, display_results, save_to_json
    
    try:
        # PDF에서 텍스트 추출
        print(f"\n📄 PDF 파일 읽기 중: {pdf_filename}")
        content = extract_text_from_pdf(pdf_filename)
        metadata = get_pdf_metadata(pdf_filename)
        
        print(f"✅ PDF 읽기 완료 - {len(content)} 문자, {metadata['pages']} 페이지")
        
        # 문서 정보
        title = metadata.get('title') or metadata.get('subject') or Path(pdf_filename).stem
        law_number = f"PDF 문서 - {pdf_filename}"
        
        # 문서 처리 (처음 3개 조항만)
        result = process_document(
            title=title,
            law_number=law_number,
            content=content,
            max_articles=3
        )
        
        # 결과 표시
        display_results(result)
        
        # JSON으로 저장
        output_filename = f"{Path(pdf_filename).stem}_result.json"
        json_path = save_to_json(result, output_filename)
        
        print("\n" + "="*80)
        print("✨ PDF 처리 완료!")
        print("="*80)
        
    except Exception as e:
        print(f"\n❌ PDF 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ PDF 파일이 업로드되지 않았습니다. 이전 셀을 실행하여 파일을 업로드하세요.")

## 6️⃣ 결과 시각화

추출된 지식 그래프를 시각화합니다.

In [ ]:
# 결과 시각화
import json
import matplotlib.pyplot as plt
from collections import Counter

# JSON 파일 읽기 (마지막으로 생성된 파일)
import glob
json_files = glob.glob("*_result.json")
if not json_files:
    print("⚠️ 결과 파일이 없습니다. 먼저 문서를 처리하세요.")
else:
    latest_json = max(json_files, key=lambda x: os.path.getmtime(x))
    print(f"📊 결과 파일: {latest_json}")
    
    with open(latest_json, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 관계 유형 분포
    relations = [t['relation'] for t in data['triplets']]
    relation_counts = Counter(relations)
    
    if relation_counts:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # 관계 유형 막대 그래프
        ax1.bar(range(len(relation_counts)), list(relation_counts.values()))
        ax1.set_xticks(range(len(relation_counts)))
        ax1.set_xticklabels(list(relation_counts.keys()), rotation=45, ha='right')
        ax1.set_title('관계 유형 분포')
        ax1.set_ylabel('개수')
        
        # 개체 통계
        entity_stats = {
            '전체 개체': len(data['entities']),
            '전체 관계': len(data['triplets']),
            '고유 관계': len(relation_counts)
        }
        ax2.bar(range(len(entity_stats)), list(entity_stats.values()))
        ax2.set_xticks(range(len(entity_stats)))
        ax2.set_xticklabels(list(entity_stats.keys()))
        ax2.set_title('추출 통계')
        ax2.set_ylabel('개수')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n📊 통계:")
        print(f"   - 추출된 개체: {len(data['entities'])}개")
        print(f"   - 추출된 관계: {len(data['triplets'])}개")
        print(f"   - 고유 관계 유형: {len(relation_counts)}개")
    else:
        print("⚠️ 추출된 관계가 없습니다.")

## 7️⃣ 결과 다운로드

처리된 결과를 JSON 파일로 다운로드합니다.

In [ ]:
# 결과 파일 다운로드
from google.colab import files
import glob

# 모든 결과 파일 찾기
json_files = glob.glob("*_result.json")

if json_files:
    print("📥 다운로드 가능한 파일:")
    for i, file in enumerate(json_files, 1):
        print(f"   {i}. {file}")
    
    # 모든 파일 다운로드
    for file in json_files:
        print(f"\n다운로드 중: {file}")
        files.download(file)
    
    print("\n✅ 다운로드 완료!")
else:
    print("⚠️ 다운로드할 결과 파일이 없습니다.")

## 📝 참고사항

### Colab 환경 제한사항
- 무료 GPU (T4) 사용 시간 제한 있음
- Gemini API 무료 tier는 분당 요청 제한이 있음
- 대용량 PDF 처리 시 메모리 부족 가능
- 처음 3개 조항만 처리하도록 제한됨 (테스트 용도)

### 더 많은 조항 처리하기
위 코드에서 `max_articles=3`을 원하는 숫자로 변경하세요. 단, API 제한을 주의하세요.

### 문제 해결
- API 키 오류: Gemini API 키를 다시 확인하세요
- 메모리 부족: 런타임을 재시작하고 더 작은 문서로 시도하세요
- GPU 오류: 런타임 유형을 "T4 GPU"로 변경하세요

### 전체 기능 사용하기
Memgraph와 더 많은 기능을 사용하려면 로컬 환경이나 Docker를 사용하세요.
자세한 내용은 [GitHub 저장소](https://github.com/WB-Jang/Knowledge-graph-construction-LLM-v2)를 참조하세요.